In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

In [20]:
train = TabularDataset('/kaggle/input/dianxin/train.csv')
test = TabularDataset('/kaggle/input/dianxin/test.csv')

Loaded data from: /kaggle/input/dianxin/train.csv | Columns = 69 / 69 | Rows = 150000 -> 150000
Loaded data from: /kaggle/input/dianxin/test.csv | Columns = 68 / 68 | Rows = 30000 -> 30000


In [21]:
data = pd.concat([train, test], axis=0, ignore_index=True)
data

,客户ID,地理区域,是否双频,是否翻新机,当前手机价格,手机网络功能,婚姻状况,家庭成人人数,信息库匹配,预计收入,...,客户生命周期内平均月费用,客户生命周期内的平均每月使用分钟数,客户整个生命周期内的平均每月通话次数,过去三个月的平均每月使用分钟数,过去三个月的平均每月通话次数,过去三个月的平均月费用,过去六个月的平均每月使用分钟数,过去六个月的平均每月通话次数,过去六个月的平均月费用,是否流失
0,0,7,0,-1,181,0,2,0,0,3,...,24,286,91,351,121,23,303,101,25,0.0
1,1,13,1,0,1399,0,3,0,0,0,...,44,447,190,483,199,40,488,202,44,1.0
2,2,14,1,0,927,0,2,4,0,6,...,48,183,79,271,95,71,209,77,54,0.0
3,3,1,0,0,232,0,3,-1,1,-1,...,42,303,166,473,226,72,446,219,65,1.0
4,4,0,-1,0,699,0,1,2,0,3,...,36,119,24,88,15,35,106,21,37,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,179995,4,0,0,173,1,2,2,0,2,...,49,309,95,457,163,97,377,136,73,NaN
179996,179996,9,0,0,150,1,1,3,0,8,...,37,106,66,52,12,28,46,15,28,NaN
179997,179997,6,1,0,1049,0,1,3,0,6,...,37,299,179,243,143,39,300,179,38,NaN
179998,179998,16,0,0,209,1,-1,-1,1,-1,...,22,58,30,12,8,13,33,16,19,NaN


In [22]:
data = pd.concat([train, test], axis=0, ignore_index=True)

features = [f for f in data.columns if f not in ['客户ID']]

train = data[data['是否流失'].notnull()].reset_index(drop=True)
test = data[data['是否流失'].isnull()].reset_index(drop=True)

x_train = train[features]
x_test = test[features]

y_train = train['是否流失']

In [23]:
x_train

,地理区域,是否双频,是否翻新机,当前手机价格,手机网络功能,婚姻状况,家庭成人人数,信息库匹配,预计收入,信用卡指示器,...,客户生命周期内平均月费用,客户生命周期内的平均每月使用分钟数,客户整个生命周期内的平均每月通话次数,过去三个月的平均每月使用分钟数,过去三个月的平均每月通话次数,过去三个月的平均月费用,过去六个月的平均每月使用分钟数,过去六个月的平均每月通话次数,过去六个月的平均月费用,是否流失
0,7,0,-1,181,0,2,0,0,3,1,...,24,286,91,351,121,23,303,101,25,0.0
1,13,1,0,1399,0,3,0,0,0,1,...,44,447,190,483,199,40,488,202,44,1.0
2,14,1,0,927,0,2,4,0,6,0,...,48,183,79,271,95,71,209,77,54,0.0
3,1,0,0,232,0,3,-1,1,-1,1,...,42,303,166,473,226,72,446,219,65,1.0
4,0,-1,0,699,0,1,2,0,3,0,...,36,119,24,88,15,35,106,21,37,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,10,1,0,1350,0,3,0,0,0,1,...,156,474,160,239,80,74,346,122,83,1.0
149996,6,1,0,542,0,3,-1,1,-1,1,...,52,968,208,1158,257,58,1307,261,57,0.0
149997,15,1,0,1300,0,1,2,0,6,0,...,39,504,205,544,203,45,531,205,47,1.0
149998,12,1,0,1399,0,4,1,0,-1,0,...,91,685,249,233,140,94,432,236,97,1.0


In [24]:
y_train

0         0.0
1         1.0
2         0.0
3         1.0
4         1.0
         ... 
149995    1.0
149996    0.0
149997    1.0
149998    1.0
149999    0.0
Name: 是否流失, Length: 150000, dtype: float64

In [26]:
x_test

,地理区域,是否双频,是否翻新机,当前手机价格,手机网络功能,婚姻状况,家庭成人人数,信息库匹配,预计收入,信用卡指示器,...,客户生命周期内平均月费用,客户生命周期内的平均每月使用分钟数,客户整个生命周期内的平均每月通话次数,过去三个月的平均每月使用分钟数,过去三个月的平均每月通话次数,过去三个月的平均月费用,过去六个月的平均每月使用分钟数,过去六个月的平均每月通话次数,过去六个月的平均月费用,是否流失
0,15,1,0,909,0,3,-1,1,-1,1,...,57,253,104,68,39,41,141,59,40,NaN
1,8,1,0,1173,0,1,1,0,5,0,...,94,886,285,1036,305,92,1064,325,94,NaN
2,7,1,0,1049,0,2,0,0,2,0,...,69,513,195,673,221,78,584,212,79,NaN
3,-1,0,0,209,1,0,1,0,7,0,...,32,47,33,72,53,33,76,54,33,NaN
4,-1,0,0,209,2,-1,-1,1,-1,-1,...,16,59,37,75,36,19,69,38,17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,4,0,0,173,1,2,2,0,2,0,...,49,309,95,457,163,97,377,136,73,NaN
29996,9,0,0,150,1,1,3,0,8,0,...,37,106,66,52,12,28,46,15,28,NaN
29997,6,1,0,1049,0,1,3,0,6,0,...,37,299,179,243,143,39,300,179,38,NaN
29998,16,0,0,209,1,-1,-1,1,-1,-1,...,22,58,30,12,8,13,33,16,19,NaN


In [28]:
predictor = TabularPredictor(label='是否流失').fit(train_data=x_train)
predictions = predictor.predict(x_test)

No path specified. Models will be saved in: "AutogluonModels/ag-20221011_074801/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20221011_074801/"
AutoGluon Version:  0.5.2
Python Version:     3.7.12
Operating System:   Linux
Train Data Rows:    150000
Train Data Columns: 67
Label Column: 是否流失
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0.0, 1.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess t

	0.6312	 = Validation score   (accuracy)
	9.24s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...
	0.636	 = Validation score   (accuracy)
	8.89s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7684	 = Validation score   (accuracy)
	129.13s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.774	 = Validation score   (accuracy)
	149.49s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: CatBoost ...
	0.6048	 = Validation score   (accuracy)
	3.26s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7264	 = Validation score   (accuracy)
	51.28s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.72	 = Validation score   (accuracy)
	52.12s	 = Training   runtime
	0.31s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.6476	 = Validation score   (accuracy)
	229.19s	 = Training   runtime
	0

[1000]	valid_set's binary_error: 0.2956
[2000]	valid_set's binary_error: 0.2688
[3000]	valid_set's binary_error: 0.2452
[4000]	valid_set's binary_error: 0.2316
[5000]	valid_set's binary_error: 0.224
[6000]	valid_set's binary_error: 0.2148
[7000]	valid_set's binary_error: 0.21
[8000]	valid_set's binary_error: 0.2048
[9000]	valid_set's binary_error: 0.2028
[10000]	valid_set's binary_error: 0.1996


	0.8004	 = Validation score   (accuracy)
	479.1s	 = Training   runtime
	2.25s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.854	 = Validation score   (accuracy)
	1.58s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 2136.41s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20221011_074801/")


In [29]:
test['是否流失'] = predictions
test[['客户ID', '是否流失']].to_csv('autogloun.csv', index=False)

In [ ]:
cv_scores.append(roc_auc_score(val_y, predictions))